In [3]:
##Preable
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import copy
import pickle
import emcee
import corner

from EightBitTransit.cTransitingImage import TransitingImage
from EightBitTransit.inversion import *
from EightBitTransit.misc import *

%matplotlib inline

In [4]:
def fillShape(grid,points): ##Pls optimize, will become a bottleneck
    
    ##This function takes a list of lists points and returns a grid with the shape bound by said points filled in.
    ##Points in the sublists are connected in the order they are given
    ##Sublists are not connected to each other
    outline = np.zeros([len(grid),len(grid[0])])##Create a grid to plot the outline on
    for i in range(len(points)):##Iterate over the list of point lists
        for j in range(len(points[i])):##Iterate over the points in each sublist
            k = (j + 1)%len(points[i])##Take also the next point, or if the point is the last point, take the first point
            outline[points[i][j][0]][points[i][j][1]] = 1 ##The points are set on the outline grid
            if points[i][j][0] - points[i][k][0] != 0: ##Fill in a line between the points as long as they have different y coordinates
                if points[i][j][0] < points[i][k][0]: ##Make sure x1,y1 is to the left of x2,y2
                    x1 = points[i][j][0]
                    y1 = points[i][j][1]
                    x2 = points[i][k][0]
                    y2 = points[i][k][1]
                else:
                    x1 = points[i][k][0]
                    y1 = points[i][k][1]
                    x2 = points[i][j][0]
                    y2 = points[i][j][1]
                for l in range(x1,x2):##For every x coordinate between the two, fill in the nearest y coordinate
                    y = int(((y1-y2)/(x1-x2))*(l - x1) + y1)
                    outline[l][y] = 1
            else:
                if points[i][j][1] < points[i][k][1]: ##Make sure x1,y1 is above x2,y2
                    x1 = points[i][j][0]
                    y1 = points[i][j][1]
                    x2 = points[i][k][0]
                    y2 = points[i][k][1]
                else:
                    x1 = points[i][k][0]
                    y1 = points[i][k][1]
                    x2 = points[i][j][0]
                    y2 = points[i][j][1]
                for l in range(y1,y2): ##For every y coordinate between the two points, fill in the nearest x coordinate
                    x = int(((x1-x2)/(y1-y2))*(l - y1) + x1)
                    outline[x][l] = 1
    ##Lines have to filled in by x and y to make sure that every column and row the shape occupies has a point on the outline
    ##Actually filling in the shape
    for i in range(len(grid)):##Iterate over the entire grid
        for j in range(len(grid[i])):
            leftPasses = 0
            rightPasses = 0
            if outline[i][j] == 1: ##All outline points are filled in
                grid[i][j] = 1
            for k in range(j):##Iterate from the current point to the left edge
                if outline[i][j-k-1] == 1 and outline[i][j-k] == 0: ##If a point is in the outline and the preceding point isn't, then the outline has been passes
                    leftPasses += 1
            for k in range(len(grid[i]) - j -1):##Repeat going toward the right edge
                if outline[i][j+k] == 1 and outline[i][j+k+1] == 0:
                    rightPasses += 1
            if (rightPasses%2==1 or leftPasses%2==1) and (leftPasses > 0 and rightPasses > 0):
                ##If all directions have passes the outline at least once and the outline has been passes an odd number of times in at least one horizontal and one vertical direction, the point is definitely within the outline
                grid[i][j] = 1
    
    return grid

In [5]:
def generatePoints(n,r,xoffset = 0,yoffset = 0,rotation = 0,proportion = 1): ##Generates points of a regular polygon with n edges, radius of r*size
    ##Size is grid size, 0 < r < 0.5, rotation rotates the shape from having a point vertically down
    points = []
    ystretch = np.sqrt(proportion)
    xstretch = 1/np.sqrt(proportion)
    for i in range(n):
        y = int(ystretch*r*np.cos(i*2*np.pi/n + np.pi/n))
        x = int(xstretch*r*np.sin(i*2*np.pi/n + np.pi/n))
        nuy = int(x*np.sin(rotation)+y*np.cos(rotation) + yoffset)
        nux = int(x*np.cos(rotation)-y*np.sin(rotation) + xoffset)
        points.append([nuy,nux])
    return points

In [6]:
def getData(filename): ##Gets the time array from the TESS Light Curve
    fp = open("./"+filename+".pkl","rb") ##Loading the file
    data = pickle.load(fp)
    fp.close()
    
    q = data[11]==0
    times = data[6][q]
    flux = data[9][q]
    err = data[10][q]
    return times, flux, err

In [7]:
def extendLC(LC, overlapTimes, times):
    extendedLC = np.ones(len(times))##Start with full flux curve
    overlapIndex = 0
    for i in range(len(times)):##Iterate over the desired time array
        if times[i] == overlapTimes[overlapIndex]: ##When the desired time has caught up to the light curve from ebt, use the ebt values
            if overlapIndex < len(overlapTimes) - 2:##Only do this to the end of the ebt light curve
                overlapIndex += 1
                extendedLC[i]  = LC[overlapIndex]
        
    return extendedLC, times

In [8]:
def model(sides,radius,rotation,stretch,velocity, tRef, times):
    points = generatePoints(sides,radius,(1/np.sqrt(stretch))*radius+1,50,rotation,stretch)
    pointsList = [points]
    planetGrid = fillShape(np.zeros([100*int(np.sqrt(stretch)),int(2*(1/np.sqrt(stretch))*radius + 2)]),pointsList)
    planet = TransitingImage(opacitymat = planetGrid, v = velocity, t_ref = tRef, t_arr = times)
    planetLC, overlapTimes = planet.gen_LC(t_arr = times)
    planetLC, time = extendLC(planetLC, overlapTimes, times)
    return planetLC

In [9]:
def logLikelihood(theta, times, flux, fluxErr):
    sides, radius, rotation, stretch, velocity, tRef = theta
    fluxPredicted = model(sides, radius,rotation,stretch,velocity,tRef,times)
    lnl = -np.sum(((flux - fluxPrediced)**2) /(2*fluxErr**2))

In [10]:
def logPrior(theta, times):
    sides, radius, rotation, stretch, velocity, tRef = theta
    lnPrior = 0
    if times[0] < tRef < times[-1]:
        lnPrior += np.log(1/(times[-1]-times[0]))
    else:
        return -np.inf
    if  0  < rotaion < 2*np.pi and 0.1 < stretch < 10:
        lnPrior += np.log(1/(2*np.pi)) + 1/2
    else:
        return -np.inf
    if 3 < sides < 24 and 0 < radius < 50 and 0 < velocity < 50:
        lnPrior += np.log(1/21) + np.log(1/50) + np.log(1/50)
    else:
        return -np.inf
    return lnPrior
        

In [11]:
def logProbability(theta, times, flux, fluxErr):
    lp = logPrior(theta, times)
    ll = logLikelihood(theta, times, flux, fluxErr)
    return lp + ll

In [ ]:
times, flux, fluxErr = getData("tesslc_13203100000000")
tRef = np.median(times)
pos = [12, 0.05, 0, 1, 1, tRef]
for i in range(5):
    pos.append(pos)

sampler = emcee.EnsembleSampler(32,6,logProbability, args=(times,flux,fluxErr))
sampler.run_mcmc(pos,5000,progress = True);